Subset data to just Detractors from Insights Survey

In [1]:
import nltk
import pandas as pd
from textblob import TextBlob
import logging
from gensim import corpora, models, similarities
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import lda
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
ltr=pd.read_csv('data/ltr2.csv')

/Users/shani16/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (5,7,10,11,13,14,15,16,17,22,24,25,26,27,29,32,33,34,35,36,39,40,41,44,45,57,58,61,68,77,78,79,80,82,107,108,144,145,146,147,154,155,156,157,158,160,161,162,163,164,166,168,171,173,174,175,176,177,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ltr.loc[ltr['Translation to English for: LTR Comment'].isnull(),'Translation to English for: LTR Comment'] = ltr['LTR Comment']

In [4]:
ltr['Survey Type'].value_counts()

Insights                    10236
Partner                        83
On Boarding (SaaS)             41
Renewal (On Premise)           18
Renewal (SaaS)                  8
On Boarding (On Premise)        3
Name: Survey Type, dtype: int64

In [5]:
ltr=ltr[(ltr['Survey Type'] == "Insights")]

In [6]:
len(ltr)

10236

In [7]:
ltr['LTR'].value_counts()

8     2624
7     2211
9     1494
10    1224
5     1005
6      932
4      233
3      222
2      137
0       95
1       59
Name: LTR, dtype: int64

In [8]:
det= ltr[(ltr['LTR'] <= 6)]

In [9]:
det['LTR'].value_counts()

5    1005
6     932
4     233
3     222
2     137
0      95
1      59
Name: LTR, dtype: int64

In [10]:
len(det)

2683

In [11]:
df=ltr[['Respondent Role', 'Translation to English for: LTR Comment', 'LTR']]

In [12]:
df.head()

,Respondent Role,Translation to English for: LTR Comment,LTR
0,End User,I would be interested they can solucuonar inco...,6
1,End User,NaN,7
2,End User,To further improved on the CA Software Bundle,8
3,End User,Network and Server Monitoring,9
5,Decision Maker,Monitoring and Service Desk,9


In [13]:
comment=pd.DataFrame(df.dropna())
comment=comment.reset_index()
comment=comment['Translation to English for: LTR Comment']
comment.columns=['role','comment', 'ltr']
comment.head()

0    I would be interested they can solucuonar inco...
1        To further improved on the CA Software Bundle
2                        Network and Server Monitoring
3                          Monitoring and Service Desk
4    Meet as soon as the so-called demands. Improve...
Name: Translation to English for: LTR Comment, dtype: object

In [14]:
documents=comment.tolist()
documents[1:5]

['To further improved on the CA Software Bundle',
 'Network and Server Monitoring',
 'Monitoring and Service Desk',
 'Meet as soon as the so-called demands. Improve the product as is requested via requests. Improving product performance.']

In [15]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords += ['.', ',', '(', ')', "'", '"', 'ca']
stopwords[:10]

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your']

In [16]:

from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words=stopwords,
                                 use_idf=True,ngram_range=(1,3), decode_error='ignore')
%time tfidf_matrix = tfidf_vectorizer.fit_transform(documents) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 938 ms, sys: 79.4 ms, total: 1.02 s
Wall time: 1.01 s
(5940, 2)


In [17]:
print tfidf_matrix

  (9, 0)	1.0
  (10, 1)	0.718755574748
  (10, 0)	0.695262845095
  (12, 1)	1.0
  (13, 1)	0.718755574748
  (13, 0)	0.695262845095
  (15, 1)	1.0
  (17, 1)	1.0
  (20, 0)	1.0
  (22, 0)	1.0
  (24, 1)	1.0
  (27, 1)	1.0
  (30, 1)	1.0
  (33, 0)	1.0
  (34, 0)	1.0
  (35, 0)	1.0
  (44, 1)	1.0
  (46, 1)	1.0
  (58, 1)	1.0
  (60, 0)	1.0
  (61, 0)	1.0
  (62, 1)	0.459180043079
  (62, 0)	0.888343226483
  (66, 1)	1.0
  (68, 0)	1.0
  :	:
  (5877, 1)	1.0
  (5880, 1)	1.0
  (5881, 0)	1.0
  (5882, 0)	1.0
  (5883, 0)	1.0
  (5884, 0)	1.0
  (5891, 1)	0.95174821006
  (5891, 0)	0.306880016697
  (5895, 0)	1.0
  (5901, 0)	1.0
  (5902, 0)	1.0
  (5903, 1)	0.718755574748
  (5903, 0)	0.695262845095
  (5908, 1)	1.0
  (5911, 1)	1.0
  (5915, 0)	1.0
  (5916, 0)	1.0
  (5920, 0)	1.0
  (5928, 0)	1.0
  (5930, 0)	1.0
  (5933, 1)	0.718755574748
  (5933, 0)	0.695262845095
  (5935, 0)	1.0
  (5936, 0)	1.0
  (5937, 0)	1.0


In [19]:
terms = tfidf_vectorizer.get_feature_names()

In [20]:
terms

[u'products', u'support']

In [28]:
from nltk.stem.porter import *

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
        return stemmed

stemmer = PorterStemmer()
stemmed = stem_tokens(documents, stemmer)
count = Counter(stemmed)
print count.most_common(100)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xff in position 42: ordinal not in range(128)

In [30]:
import nltk
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

token_dict = {}
stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems


for file in files:
    shakes = documents
    text = shakes.read()
    lowers = text.lower()
        no_punctuation = lowers.translate(None, string.punctuation)
        token_dict[file] = no_punctuation
        
#this can take some time
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

NameError: name 'path' is not defined

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print
print

In [ ]:
dist

In [ ]:
from sklearn import decomposition
num_topics = 10
num_top_words = 20
clf = decomposition.NMF(n_components=num_topics, random_state=1)

In [ ]:
doctopic = clf.fit_transform(documents)

In [ ]:
comment_new[254:256]

In [ ]:
data= [i.decode("utf-8") for i in documents]


In [ ]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"', 'ca']

In [ ]:
vectorizer = CountVectorizer(min_df=1, decode_error='ignore')
X=vectorizer.fit_transform(documents)
X=X.toarray()
print X

In [ ]:
X.shape

In [ ]:
test(comment)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
test(comment)

In [ ]:

comment_new=[]
for i in documents:
     comment_new.append(i.encode('utf-8').strip())

In [ ]:
def encode(x):
    x=x.decode()
    x=[w for w in x.split() if w not in stop]
    return (' ').join(x)


In [ ]:
comment_clean=[]
for i in comment:
    t=unicode(i, "utf-8", errors = "ignore") 
    comment_clean.append(t)

In [ ]:
comment_clean[0:14]

In [ ]:
comment_clean

In [ ]:
for i in comment:
    t=encode('utf-8', 'replace')
    comment_clean.append(t)   

In [ ]:
documents.encode('utf-8').strip()

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english',
                decode_error='ignore')


X = vectorizer.fit_transform(comment)

In [ ]:
test=str(comment_clean)

In [ ]:
X=X.toarray()

In [ ]:
X

In [ ]:
X.shape

In [ ]:
import numpy as np
import lda
# X = lda.datasets.load_reuters()
X.shape
model = lda.LDA(n_topics=3, n_iter=500, random_state=1)
model.fit(X)
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

In [ ]:
bag_of_words.toarray()

In [ ]:
X=bag_of_words.toarray()

In [ ]:
test= str(documents).strip('[]')

In [ ]:
test

In [ ]:
import numpy as np
import lda
import lda.datasets
# X = lda.datasets.load_reuters()
# vocab = lda.datasets.load_reuters_vocab()
# titles = lda.datasets.load_reuters_titles()
X.shape
model = lda.LDA(n_topics=20, n_iter=500, random_state=1)
model.fit(X)
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8


In [ ]:
topic_word

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bag_of_words = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False).fit_transform(t)

In [ ]:
bag_of_words.toarray()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

AC_clusters = AgglomerativeClustering(n_clusters=50, affinity='cosine', linkage='average').fit_predict(bag_of_words.toarray())

sorted_clu_dict = text_utils.rank_clusters(AC_clusters)

print 'Top clusters:'
pprint(sorted_clu_dict[:10])

In [ ]:
import nltk

def dialogue_act_features(test):
    """
        Extracts a set of features from a message.
    """
    features = {}
    tokens = nltk.word_tokenize(test)
    for t in tokens:
        features['contains(%s)' % t.lower()] = True    
    return features

In [ ]:
dialogue_act_features(test[0])

In [ ]:
for i in range(1,len(test)):
    test[i].decode('ascii')

In [ ]:
len(test)

In [ ]:
import nltk
import string

from collections import Counter

def get_tokens():
   with open('/opt/datacourse/data/parts/shakes-1.txt', 'r') as shakes:
    text = shakes.read()
    lowers = text.lower()
    #remove the punctuation using the character deletion step of translate
    no_punctuation = lowers.translate(None, string.punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

tokens = get_tokens()
count = Counter(tokens)
print count.most_common(10)

In [ ]:
TextBlob(str(test[0])).sentiment

In [ ]:
test1

In [ ]:
test1.tags

In [ ]:
test= str(test).strip('[]')

In [ ]:
for item in test:
    print item.polarity
    

In [ ]:
for item in test:
...     print item.replace('\n', ' ')


In [ ]:
type(test[0])

In [ ]:
for item in test.sentences:
...   print item.sentiment.polarity